In [9]:
# import json
import requests
from datetime import datetime, timedelta


In [14]:
# From openuv 
api_key = "openuv-4olrlo0dow96-io"

# headers for api call
headers = {
    "x-access-token": api_key,
    "Content-Type": "application/json"
}
# Set necessary variables
lat = 34.11
lng = -117.70
alt = 100

# Determine time zone
from zoneinfo import ZoneInfo
utc = ZoneInfo('UTC')
localtz = ZoneInfo('America/Los_Angeles')
database_time = datetime.utcnow()

utctime = database_time.replace(tzinfo=utc)
localtime = utctime.astimezone(localtz)
print(localtime)

# Create url with api key above

url = "https://api.openuv.io/api/v1/forecast?lat={}&lng={}&alt={}&dt=".format(lat,lng,alt)

request = requests.get(url= url, headers=headers)

# Part II: Store data in json file
data = request.json()


# Create lists for time and uv data to be stored in
timestamps = []
uv_data = []

# Loop through results to add data to lists
for element in data['result']:
    # format_datetime = datetime.strptime(element['uv_time'], '%Y-%m-%dT%H:%M:%S.%f%z')
    # format_time = datetime.strftime(format_datetime, "%H:%M")
    # timestamps.append(format_time)
    
    timestamps.append(localtime.strftime("%H"))
    localtime += timedelta(hours=1)
    uv_data.append(element['uv'])

print(timestamps)
print(uv_data)




2023-11-01 14:00:12.522858-07:00
['14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '00']
[0, 0.2242, 0.8765, 1.8651, 2.9251, 3.608, 3.4347, 2.762, 1.6103, 0.7134, 0.1529]
